In [248]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [249]:
tokenizer1 = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
tokenizer2 = AutoTokenizer.from_pretrained("google/mt5-small", model_max_length=4096)

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [250]:
text = 'Givet følgende titler og undertekster fra tidligere læste artikler:\nArtikel 1:\nTitel: Wagner-chef afviser russiske meldinger om fremgang\nUndertekst: Ruslands forsvarsministerium tager ifølge Jevgenij Prigozjin fejl med dets påstande om fremgang i Bakhmut\nArtikel 2:\nTitel: Har du råd? Sænker prisen på sit \'slot\' med millioner\nUndertekst: Konen efter afdøde Jørgen Haugen Sørensen, der var en af landets mest betydningsfulde billedhuggere, har sat ægteparrets fælles gigantiske hus i Italien på markedet.\nArtikel 3:\nTitel: Så klappede fælden: Techsvindler skal bag tremmer\nUndertekst: Elizabeth Holmes slipper ikke for fængsel, mens hun afventer appelsag, slår en domstol fast\nArtikel 4:\nTitel: Slår op med kæresten\nUndertekst: Billie Eilish er nu single igen\nArtikel 5:\nTitel: Idømt fængsel og udvisning: Skyldig i psykisk vold og voldtægt\nUndertekst: En 56-årig mand er fundet skyldig i en lang række forhold, herunder trusler, psykisk vold og voldtægt. Han har anket dommen\nEr brugeren sandsynligvis interesseret i at klikke på den følgende artikel:\n Titel: SSI: Tegn på stigning i tilfælde af skoldkopper\n Undertekst: Det tyder ifølge SSI på forhøjet forekomst af skoldkopper i 2022 og 2023 efter år med lav smitte under corona? (ja/nej)'

In [251]:
a = tokenizer1(text, return_tensors='pt', padding='max_length', max_length=1024, truncation=True)

In [252]:
a.attention_mask.shape

torch.Size([1, 1024])

In [253]:
input_text = "Translate German to English: ja ja ja"
output1 = tokenizer1("hallo", return_tensors="pt")
output2 = tokenizer2("hallo", return_tensors="pt", padding='max_length', max_length=4096, truncation=True)
output1, output2

({'input_ids': tensor([[18691,   268,     1]]), 'attention_mask': tensor([[1, 1, 1]])},
 {'input_ids': tensor([[18691,   268,     1,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])})

In [254]:
output1.attention_mask.shape, output2.attention_mask.shape

(torch.Size([1, 3]), torch.Size([1, 4096]))

In [255]:
decoder_input_ids1 = tokenizer1("", return_tensors="pt").input_ids
decoder_input_ids2 = tokenizer2("", return_tensors="pt", padding='max_length', max_length=4096, truncation=True).input_ids
decoder_input_ids1, decoder_input_ids2

(tensor([[1]]), tensor([[1, 0, 0,  ..., 0, 0, 0]]))

In [256]:
decoder_attention_mask1 = tokenizer1("", return_tensors="pt").attention_mask
decoder_attention_mask2 = tokenizer2("", return_tensors="pt", padding='max_length', max_length=4096, truncation=True).attention_mask
decoder_attention_mask1, decoder_attention_mask2

(tensor([[1]]), tensor([[1, 0, 0,  ..., 0, 0, 0]]))

In [257]:
outputs1 = model(input_ids=output1["input_ids"], attention_mask=output1["attention_mask"], decoder_input_ids=decoder_input_ids1, decoder_attention_mask=decoder_attention_mask1)
outputs2 = model(input_ids=output2["input_ids"], attention_mask=output2["attention_mask"], decoder_input_ids=decoder_input_ids2, decoder_attention_mask=decoder_attention_mask2)

In [70]:
outputs1.logits.shape, outputs2.logits.shape

(torch.Size([1, 1, 250112]), torch.Size([1, 4096, 250112]))

In [60]:
outputs1.logits[0,0,36339], outputs1.logits[0,0,375]

(tensor(-69.3544, grad_fn=<SelectBackward0>),
 tensor(-62.9657, grad_fn=<SelectBackward0>))

In [34]:
outputs2.logits[0,0,36339], outputs2.logits[0,0,375]

(tensor(-69.3544, grad_fn=<SelectBackward0>),
 tensor(-62.9657, grad_fn=<SelectBackward0>))

In [57]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [71]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [86]:
input_texts = ["Translate English to German: How are you?????  ---   ???? //??? ? ???", "Translate English to French: I am fine."]
inputs = tokenizer(input_texts, return_tensors="pt", padding=True)


In [109]:
decoder_start = [" 3 d", "ja"]
decoder_inputs = tokenizer(decoder_start, return_tensors="pt", padding=True)

In [110]:
decoder_inputs.input_ids

tensor([[381, 331,   1],
        [432,   1,   0]])

In [111]:
inputs.input_ids

tensor([[89349,  5413,   288, 20567,   267,  5100,   418,   521, 57185, 25976,
         27513,  1337,  5212,   259,   291, 22443,     1],
        [89349,  5413,   288, 21273,   267,   336,   728,  7495,   260,     1,
             0,     0,     0,     0,     0,     0,     0]])

In [116]:
model.generate(inputs.input_ids, attention_mask=inputs.attention_mask, decoder_start_token_id=decoder_inputs.input_ids, max_length=2)

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of decoder_input_ids is 3, but `max_length` is set to 2. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


tensor([[   381,    331,      1, 250099],
        [   432,      1,      0, 250099]])

In [114]:
inputs.attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])

In [112]:
a=model(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask, decoder_input_ids=decoder_inputs.input_ids, decoder_attention_mask=decoder_inputs.attention_mask)

In [113]:
a.logits.shape

torch.Size([2, 3, 250112])

In [155]:
from transformers import MT5ForQuestionAnswering, AutoTokenizer

model = MT5ForQuestionAnswering.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MT5ForQuestionAnswering were not initialized from the model checkpoint at google/mt5-small and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would hav

In [158]:
inputs["labels"]

tensor([[15570,   261,   609,  3031,   260,     1]])

In [159]:
from transformers import MT5ForQuestionAnswering, MT5Tokenizer
import torch

In [161]:
# Load the model and tokenizer
model = MT5ForQuestionAnswering.from_pretrained("google/mt5-small")
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

Some weights of MT5ForQuestionAnswering were not initialized from the model checkpoint at google/mt5-small and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [173]:
context = "The capital of France is Paris."
question = "What is the capital of France?"
input_text = f"question: {question} context: {context}"

inputs = tokenizer(input_text, return_tensors="pt")

outputs = model(**inputs)

In [174]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Find the start and end positions of the answer
start_position = torch.argmax(start_logits, dim=1).item()
end_position = torch.argmax(end_logits, dim=1).item()

# Decode the answer
input_ids = inputs["input_ids"][0]
answer_ids = input_ids[start_position:end_position + 1]
answer = tokenizer.decode(answer_ids)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What is the capital of France?
Answer: 


In [147]:
outputs.logits.shape

torch.Size([1, 3, 250112])

In [148]:
outputs.logits[0,0,36339], outputs.logits[0,0,375]

(tensor(-41.9595, grad_fn=<SelectBackward0>),
 tensor(-39.8989, grad_fn=<SelectBackward0>))

In [149]:
tokenizer.decode(outputs.logits[0].argmax(dim=-1))

'<extra_id_0>𓆡 <extra_id_14>'

In [154]:
outputs.logits.shape

torch.Size([1, 3, 250112])

In [175]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [222]:
input = "Did astronauts find life on Mars?"
target = "No"
inputs = tokenizer(input, text_target=target, return_tensors="pt")

outputs = model(**inputs)
loss = outputs.loss

In [208]:
outputs.logits.shape

torch.Size([1, 2, 250112])

In [223]:
outputs.logits[0,0,36339], outputs.logits[0,0,375]

(tensor(-41.4339, grad_fn=<SelectBackward0>),
 tensor(-40.0831, grad_fn=<SelectBackward0>))

In [224]:
outputs.logits[0,-1,36339], outputs.logits[0,-1,375]

(tensor(-12.8843, grad_fn=<SelectBackward0>),
 tensor(-7.9506, grad_fn=<SelectBackward0>))

In [201]:
predicted_token_ids = torch.argmax(outputs.logits, dim=-1)
generated_text = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
print(generated_text)

['<extra_id_0>XzFf']


In [216]:
outputs.loss

tensor(23.2801, grad_fn=<NllLossBackward0>)

In [245]:
article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
summary = "ja"
inputs = tokenizer(article, text_target=summary, return_tensors="pt")

outputs = model(**inputs)
loss = outputs.loss

In [246]:
outputs.logits.shape

torch.Size([1, 2, 250112])

In [ ]:
model.decoder

In [247]:
outputs.logits[0,-1,36339], outputs.logits[0,-1,375]

(tensor(-13.4170, grad_fn=<SelectBackward0>),
 tensor(-24.0090, grad_fn=<SelectBackward0>))

In [263]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
summary = "Weiter Verhandlung in Syrien,"
inputs = tokenizer(article, text_target=summary, return_tensors="pt")

outputs = model(**inputs)
logits = outputs.logits

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [261]:
loss

tensor(14.2996, grad_fn=<NllLossBackward0>)

In [262]:
outputs.logits

tensor([[[-80.9701, -39.9612, -78.7373,  ..., -76.0993, -80.1216, -80.3935],
         [-43.3847,  -3.8463, -42.7037,  ..., -41.5528, -43.0312, -43.3387],
         [-53.3445,  -6.6598, -52.1731,  ..., -50.8407, -52.4811, -53.4101],
         ...,
         [-74.5119, -22.3036, -72.7793,  ..., -70.1930, -73.6492, -74.9771],
         [-65.6037, -14.4813, -64.3154,  ..., -62.1984, -64.3051, -65.2954],
         [-56.1277,  -3.4734, -55.2107,  ..., -53.1906, -55.2984, -56.2820]]],
       grad_fn=<UnsafeViewBackward0>)

In [265]:
outputs.logits.shape

torch.Size([1, 7, 250112])

In [266]:
# Example data
examples = [
    {"input": "Is it raining?", "target": "yes"},
    {"input": "Is the sky green?", "target": "no"}
]

# Tokenize the inputs and targets
inputs = tokenizer([ex["input"] for ex in examples], return_tensors="pt", padding=True, truncation=True)
with tokenizer.as_target_tokenizer():
    targets = tokenizer([ex["target"] for ex in examples], return_tensors="pt", padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [267]:
outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=targets["input_ids"])
    
# Custom NLL loss
logits = outputs.logits
shift_logits = logits[..., :-1, :].contiguous()
shift_labels = targets["input_ids"][..., 1:].contiguous()
loss_fct = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

In [274]:
shift_logits.shape

torch.Size([2, 1, 250112])

In [269]:
shift_labels

tensor([[1],
        [1]])

In [273]:
targets['input_ids']

tensor([[36339,     1],
        [  375,     1]])

In [275]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [294]:
# Example data
examples = [
    {"input": "Is it raining?", "target": "ja"},
    {"input": "Is the sky green?", "target": "no"}
]

# Tokenize the inputs and targets
inputs = tokenizer([ex["input"] for ex in examples], return_tensors="pt", padding=True, truncation=True)
with tokenizer.as_target_tokenizer():
    targets = tokenizer([ex["target"] for ex in examples], return_tensors="pt", padding=True, truncation=True)

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [295]:
inputs

{'input_ids': tensor([[ 2485,   609, 47637,   347,   291,     1],
        [ 2485,   287, 20495, 16263,   291,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

In [296]:
targets

{'input_ids': tensor([[36339, 36339,     1],
        [  375,   375,     1]]), 'attention_mask': tensor([[1, 1, 1],
        [1, 1, 1]])}

In [297]:
outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=targets["input_ids"])

In [298]:
outputs.logits.shape

torch.Size([2, 3, 250112])

In [299]:
outputs.logits.argmax(dim=-1), outputs.logits.max(dim=-1)

(tensor([[250099, 250099, 250099],
         [250099, 250099, 250099]]),
 torch.return_types.max(
 values=tensor([[-13.6774, -49.1502, -24.4767],
         [-14.5246, -39.1459, -51.4582]], grad_fn=<MaxBackward0>),
 indices=tensor([[250099, 250099, 250099],
         [250099, 250099, 250099]])))

In [291]:
tokenizer.decode(outputs.logits.argmax(dim=-1)[0]), tokenizer.decode(outputs.logits.argmax(dim=-1)[1])

('<extra_id_0> <extra_id_0>', '<extra_id_0> <extra_id_0>')

In [293]:
tokenizer.pad

<bound method PreTrainedTokenizerBase.pad of T5TokenizerFast(name_or_path='google/mt5-small', vocab_size=250100, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}>